## Import des librairies 

In [1]:
import boto3
import os
from skimage import io
from sklearn import decomposition
import numpy as np
import pandas as pd
from PIL import Image
from pyspark import SparkContext
from skimage.color import rgb2gray
import pyspark
import requests
from io import BytesIO
import tempfile
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
df = pd.DataFrame()

In [3]:
for file in os.listdir("fruits-360/Training/avocado"):
    nvlle_ligne = pd.Series("avocado")
    nvlle_ligne = nvlle_ligne.append(pd.Series(list(Image.open(f"fruits-360/Training/avocado/{file}").tobytes())), ignore_index=True)
    df = df.append(nvlle_ligne, ignore_index=True)

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,29991,29992,29993,29994,29995,29996,29997,29998,29999,30000
0,avocado,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
1,avocado,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
2,avocado,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
3,avocado,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
4,avocado,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0


In [5]:
pca = decomposition.PCA(n_components=100)

In [6]:
pca.fit(df.drop(columns=[0]))

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [7]:
pca.explained_variance_ratio_.cumsum()

array([0.55579395, 0.76278481, 0.79830406, 0.82911431, 0.85367885,
       0.87724746, 0.89200928, 0.90507435, 0.91454122, 0.92360102,
       0.93064434, 0.93597161, 0.94057347, 0.94423668, 0.94764555,
       0.95066038, 0.95327063, 0.95563125, 0.95767199, 0.95936057,
       0.96088702, 0.96236476, 0.96375244, 0.96508009, 0.96626801,
       0.96741665, 0.96847306, 0.96951302, 0.9704978 , 0.97142405,
       0.9722372 , 0.97303024, 0.97379095, 0.97451862, 0.97519095,
       0.97583777, 0.97643775, 0.97696526, 0.97747195, 0.97797068,
       0.97843426, 0.97888965, 0.97932515, 0.97975483, 0.98016526,
       0.98055255, 0.98093384, 0.98129341, 0.9816434 , 0.98198289,
       0.98231033, 0.98261587, 0.98291253, 0.98319806, 0.98347821,
       0.98375388, 0.98401895, 0.98427923, 0.98453211, 0.98477836,
       0.98501509, 0.98524613, 0.98547289, 0.98568909, 0.98589989,
       0.98610723, 0.98631173, 0.9865082 , 0.98669879, 0.98688687,
       0.98706794, 0.98724707, 0.98742211, 0.98758366, 0.98774

In [8]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.title("Eboulis des valeurs propres")
plt.plot(range(1,101),pca.explained_variance_ratio_)
plt.xlabel("valeurs propres")
plt.ylabel("Taux de variance expliquée")
plt.show

fig = plt.figure()
plt.title("Cumul éboulis des valeurs propres")
plt.plot(range(1,101),pca.explained_variance_ratio_.cumsum())
plt.xlabel("valeurs propres")
plt.ylabel("Taux de variance expliquée cumulée")

Text(0, 0.5, 'Taux de variance expliquée cumulée')

In [9]:
np.where(pca.explained_variance_ratio_.cumsum()>0.95)

(array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
        32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
        49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
        66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82,
        83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
       dtype=int64),)

## Initialisation de l'environnement

In [11]:
spark = SparkSession.builder.master("local[1]").appName("Sparkexemple2").getOrCreate()

In [12]:
s3 = boto3.resource('s3')

In [13]:
bucket = s3.Bucket('apocp8')

## Définition des fonctions utiles à l'application

In [14]:
def categorie(liste):
    #Fonction permettant de récuperer le label de l'image
    l = []
    for element in liste:
        categ = element.split('/')
        categ = categ[-2]
        l.append(categ)
    return l

In [15]:
def url(liste):
    #Fonction permettant d'obtenir l'URL d'une image a partir de sa clef Amazon S3
    l = []
    location = boto3.client('s3').get_bucket_location(Bucket='apocp8')['LocationConstraint']
    for element in liste:
        rn = element.split(" ")
        rn = "+".join(rn)
        URL = "https://s3-%s.amazonaws.com/%s/%s" % (location, 'apocp8', element)
        l.append(URL)
    return l

In [16]:
def telechargement_img(url):
    #Fonction permettant de télécharger une image sous format np.array depuis une URL
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = np.array(img)
    return img

In [17]:
def chemin_sauvegarde(string):
    #Fonction permettant de creer la clef Amazon S3 des fichiers à enregistrer
    chemin = string.split('/')
    chemin[-3] = 'transforme'
    key = chemin[-3:]
    key = "/".join(key)
    key = key.split('.')
    key[-1]=('npy')
    key = ".".join(key)
    return key

In [18]:
def upload (key,fichier):
    #Fonction permettant l'upload des fichier sur Amazon S3
    ftmp = tempfile.NamedTemporaryFile()
    fname = ftmp.name + ".npy"
    np.save(fname, fichier, allow_pickle=False)
    out = np.load(fname, allow_pickle=False)    
    s3 = boto3.resource('s3')
    s3.Object('apocp8', key).upload_file(fname)

In [19]:
def array_to_string(liste):
    return '[' + ','.join([str(elem) for elem in liste]) + ']'

## Création des données

In [20]:
liste_img=[]
for obj in bucket.objects.all():
    #Récupération des clef des images de label "Apple Braeburn"
    if 'Training/Apple Braeburn' in obj.key:
        liste_img.append(obj.key)

In [21]:
len(liste_img)

492

In [ ]:
l_categ = categorie(liste_img)

In [23]:
liste_url = url(liste_img)

In [24]:
data_image = []
for i in range(len(l_categ)):
    #Création d'une liste de tuples contenant le label et l'URL des images
    data_image.append((l_categ[i],liste_url[i]))

## Traitement des images

In [25]:
rdd=spark.sparkContext.parallelize(data_image)

In [26]:
rdd2 = rdd.map(lambda x: (x[0], x[1], telechargement_img(x[1])))

In [27]:
rdd3 = rdd2.map(lambda x:(x[0],x[1],rgb2gray(x[2])))

In [28]:
rdd4 = rdd3.map(lambda x:(x[0],x[1],x[2],chemin_sauvegarde(x[1])))

In [30]:
rdd4.map(lambda x:(x[3],upload(x[3],x[2]))).count()

492

## Mise en place d'une ACP

In [31]:
rdd_acp = rdd2.map(lambda x: (x[0], Vectors.dense(x[2].astype("int").flatten().tolist())))

In [32]:
schema = ["label", "features"]

In [33]:
df = spark.createDataFrame(rdd_acp, schema=schema)

In [34]:
pca = PCA(k=20, inputCol="features", outputCol="pca_features")

In [35]:
model = pca.fit(df)
#Erreur out of memory

Py4JJavaError: An error occurred while calling o72.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7) (Sam-PC executor driver): java.lang.OutOfMemoryError: Java heap space
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:194)
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:191)
	at breeze.linalg.DenseVector$mcD$sp.<init>(DenseVector.scala:67)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.$anonfun$computeDenseVectorCovariance$1(RowMatrix.scala:160)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix$$Lambda$2221/1770486258.apply(Unknown Source)
	at scala.collection.TraversableOnce.$anonfun$foldLeft$1(TraversableOnce.scala:162)
	at scala.collection.TraversableOnce.$anonfun$foldLeft$1$adapted(TraversableOnce.scala:162)
	at scala.collection.TraversableOnce$$Lambda$133/1360657223.apply(Unknown Source)
	at s

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 57773)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 313, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 344, in process_request
    self.finish_request(request, client_address)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 357, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 717, in __init__
    self.handle()
  File "C:\apps\spark-3.1.1-bin-hadoop2.7\python\pyspark\accumulators.py", line 262, in handle
    poll(accum_updates)
  File "C:\apps\spark-3.1.1-bin-hadoop2.7\python\pyspark\accumulators.py", line 235, in poll
    if func():
  File "C:\apps\spark-3.1.1-bin-hadoop2.7\python\pyspark\accumulators.py", line 239, in accum_updates
    num_

In [ ]:
df = model.transform(df).drop("features")

In [ ]:
expl_var = model.explainedVariance.toArray()

In [ ]:
array_to_string_udf = udf(array_to_string, StringType())

In [ ]:
df = df.withColumn("features_as_string", array_to_string_udf(df["pca_features"]))

In [ ]:
df = df.drop("pca_features")

In [ ]:
df.write.csv("s3a://apocp8/dataframe_features")